# OLMoE Full Routing Experiments - Two-Phase Analysis

**Complete framework for analyzing and modifying OLMoE expert routing**

This notebook runs on:
- ✅ **Google Colab** (Recommended - GPU required)
- ✅ Local Jupyter with GPU

---

## 🚀 Quick Start (Google Colab)

1. Upload this notebook to Google Drive
2. Open with Google Colab
3. Enable GPU: `Runtime → Change runtime type → GPU → A100`
4. Run all cells

---

## 🔬 Two-Phase Experimental Approach

### **Phase 1: Baseline Analysis**
Understand the model's **natural routing behavior**:
- Which experts does it prefer?
- How concentrated is the routing?
- What's the expert utilization rate?
- Save **internal router_logits** for each sample

### **Phase 2: Modified Routing**
Test **custom routing strategies**:
- Uniform routing (equal weights)
- Normalized routing (renormalized probabilities)
- Save **internal router_logits** for comparison

### **Phase 3: Comparative Analysis**
Direct comparison:
- Does custom routing improve quality?
- What's the speed-quality trade-off?
- Generate detailed reports and visualizations

---

## 📁 Output Structure

```
two_phase_experiment/
├── logs/
│   ├── 8experts_baseline_wikitext.json                    # Summary metrics
│   ├── 8experts_baseline_wikitext_internal_routing.json  # FULL router_logits logs
│   ├── 8experts_uniform_wikitext.json
│   ├── 8experts_uniform_wikitext_internal_routing.json   # FULL router_logits logs
│   └── ... (one pair per configuration)
├── visualizations/
│   └── two_phase_comparison.png                           # 6-panel comparison
├── two_phase_results.csv                                  # All results
├── two_phase_results.json
└── two_phase_report.md                                    # Detailed analysis
```

**Each configuration gets TWO files:**
1. `{config}_internal_routing.json` - Full router_logits for all samples/layers
2. `{config}.json` - Summary metrics (perplexity, accuracy, etc.)

---

## Table of Contents

1. [Environment Setup](#1-environment-setup)
2. [GPU Configuration](#2-gpu-configuration)
3. [Installation](#3-installation)
4. [Framework Setup](#4-framework-setup)
5. [Run Full Two-Phase Experiment](#5-run-full-two-phase-experiment)
6. [Analyze Results](#6-analyze-results)
7. [Visualizations](#7-visualizations)
8. [View Internal Routing Logs](#8-view-internal-routing-logs)

---

## 1. Environment Setup

In [ ]:
import sys
import os

# Detect environment
IN_COLAB = 'google.colab' in sys.modules

print(f"Running in Google Colab: {IN_COLAB}")
print(f"Python version: {sys.version}")

# Set working directory
if IN_COLAB:
    from google.colab import drive
    print("\n📁 Mounting Google Drive...")
    drive.mount('/content/drive')
    
    WORK_DIR = '/content/drive/MyDrive/olmoe_full_experiments'
    REPO_DIR = '/content/drive/MyDrive/MOE-with-feature-selection'
else:
    WORK_DIR = './olmoe_full_experiments'
    REPO_DIR = None

os.makedirs(WORK_DIR, exist_ok=True)
os.chdir(WORK_DIR)
print(f"\n✅ Working directory: {os.getcwd()}")

if IN_COLAB:
    print(f"✅ Repository location: {REPO_DIR}")

## 2. GPU Configuration

In [ ]:
import torch

print("=" * 70)
print("GPU CONFIGURATION")
print("=" * 70)

if torch.cuda.is_available():
    print(f"\n✅ CUDA Available")
    print(f"   CUDA Version: {torch.version.cuda}")
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    
    device = 'cuda'
    torch.cuda.empty_cache()
else:
    print("\n❌ GPU not available!")
    print("\n⚠️  This notebook requires a GPU.")
    if IN_COLAB:
        print("   Enable GPU: Runtime → Change runtime type → T4/A100 GPU")
    raise Exception("GPU required for this experiment")

print(f"\n✅ Device: {device}")
print("=" * 70)

## 3. Installation

In [ ]:
%%bash
pip install -q torch transformers datasets pandas numpy matplotlib seaborn tqdm rich
echo "✅ All packages installed!"

In [ ]:
import transformers
import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

print("Package Versions:")
print(f"  torch: {torch.__version__}")
print(f"  transformers: {transformers.__version__}")
print(f"  datasets: {datasets.__version__}")
print(f"  pandas: {pd.__version__}")
print(f"  numpy: {np.__version__}")
print("\n✅ All imports successful!")

## 4. Framework Setup

In [ ]:
print("=" * 70)
print("FRAMEWORK SETUP")
print("=" * 70)

if IN_COLAB:
    # Check if repo exists in Drive
    if os.path.exists(REPO_DIR):
        print(f"\n📂 Repository exists in Google Drive")
        print(f"   Location: {REPO_DIR}")
        print(f"\n   Pulling latest changes...")
        !cd {REPO_DIR} && git pull
    else:
        print("\n📥 Cloning repository to Google Drive...")
        !git clone https://github.com/aliabbasjaffri/MOE-with-feature-selection.git {REPO_DIR}
        
    framework_dir = REPO_DIR
else:
    framework_dir = os.path.abspath('.')

# Add to Python path
if framework_dir not in sys.path:
    sys.path.insert(0, framework_dir)
    print(f"\n✅ Added to path: {framework_dir}")

# Verify framework file
framework_file = os.path.join(framework_dir, 'olmoe_routing_experiments.py')
if os.path.exists(framework_file):
    file_size = os.path.getsize(framework_file)
    print(f"✅ Found: olmoe_routing_experiments.py ({file_size:,} bytes)")
else:
    raise Exception("Framework file not found!")

print("\n" + "=" * 70)
print("✅ FRAMEWORK READY")
print("=" * 70)

In [ ]:
# Import framework
if 'olmoe_routing_experiments' in sys.modules:
    del sys.modules['olmoe_routing_experiments']

from olmoe_routing_experiments import (
    RoutingExperimentRunner,
    ModelPatchingUtils
)

print("✅ Framework imported successfully!")

## 5. Run Full Two-Phase Experiment

### Configuration

**Expert Counts:** [4, 8, 16, 32, 64]  
**Datasets:** ['wikitext', 'lambada', 'hellaswag']  
**Samples:** 500 per dataset  
**Routing Modifications:** ['uniform', 'normalized']  

**Total Experiments:** 5 expert counts × 3 datasets × 3 strategies (baseline + 2 modifications) = **45 experiments**

**Each experiment generates:**
- Summary JSON with metrics
- **Internal routing JSON with router_logits for all samples**

**Estimated Time:** ~90-120 minutes on A100 GPU

---

In [ ]:
print("=" * 70)
print("FULL TWO-PHASE ROUTING EXPERIMENT")
print("=" * 70)
print("\nConfiguration:")
print("  Expert counts: [4, 8, 16, 32, 64]")
print("  Datasets: [wikitext, lambada, hellaswag]")
print("  Samples: 500 per dataset")
print("  Modifications: [uniform, normalized]")
print("\nFeatures:")
print("  ✅ Phase 1: Baseline analysis with internal router_logits")
print("  ✅ Phase 2: Modified routing with internal router_logits")
print("  ✅ Phase 3: Comparative analysis and visualizations")
print("  ✅ Unique log file per configuration with FULL routing data")
print("\nEstimated time: ~90-120 minutes")
print("=" * 70)

# Create runner
runner = RoutingExperimentRunner(
    model_name="allenai/OLMoE-1B-7B-0924",
    device=device,
    output_dir="./two_phase_full_experiment"
)

# Run full two-phase experiment
results_df, routing_insights = runner.run_two_phase_experiment(
    expert_counts=[4, 8, 16, 32, 64],
    datasets=['wikitext', 'lambada', 'hellaswag'],
    max_samples=500,
    routing_modifications=['uniform', 'normalized']
)

print("\n" + "=" * 70)
print("✅ EXPERIMENT COMPLETE!")
print("=" * 70)
print(f"\nResults saved to: {runner.output_dir}")
print(f"\nGenerated files:")
print(f"  • 45 summary JSON files (logs/*.json)")
print(f"  • 45 internal routing JSON files (logs/*_internal_routing.json)")
print(f"  • Comparison visualizations (visualizations/two_phase_comparison.png)")
print(f"  • Detailed markdown report (two_phase_report.md)")
print(f"  • CSV results (two_phase_results.csv)")

## 6. Analyze Results

### Phase 1: Baseline Insights

In [ ]:
print("=" * 70)
print("PHASE 1: BASELINE ROUTING INSIGHTS")
print("=" * 70)

# Display baseline insights
insights_data = []
for key, insight in routing_insights.items():
    insights_data.append({
        'Experts': insight['num_experts'],
        'Dataset': insight['dataset'],
        'Perplexity': f"{insight['baseline_perplexity']:.2f}",
        'Accuracy': f"{insight['baseline_accuracy']:.4f}",
        'Experts Used': f"{insight['unique_experts_used']}/64",
        'Utilization': f"{insight['expert_utilization']:.1%}",
        'Entropy': f"{insight['avg_entropy']:.3f}"
    })

insights_df = pd.DataFrame(insights_data)
print("\n")
print(insights_df.to_string(index=False))

print("\n\n📊 Key Observations:")
avg_util = np.mean([i['expert_utilization'] for i in routing_insights.values()])
avg_entropy = np.mean([i['avg_entropy'] for i in routing_insights.values()])

print(f"  • Average expert utilization: {avg_util:.1%}")
print(f"  • Average routing entropy: {avg_entropy:.3f}")

if avg_util < 0.5:
    print(f"  ⚠️  Many experts underutilized!")
    print(f"      → Custom routing may help balance usage")

if avg_entropy < 1.0:
    print(f"  ⚠️  Routing is highly concentrated!")
    print(f"      → Model heavily favors certain experts")

### Phase 2 & 3: Comparative Analysis

In [ ]:
print("=" * 70)
print("ROUTING COMPARISON: BASELINE VS MODIFIED")
print("=" * 70)

# For each configuration, compare baseline vs modified
for num_experts in sorted(results_df['num_experts'].unique()):
    for dataset in results_df['dataset'].unique():
        print(f"\n{'='*70}")
        print(f"Configuration: {num_experts} Experts | Dataset: {dataset}")
        print(f"{'='*70}")
        
        config_df = results_df[
            (results_df['num_experts'] == num_experts) & 
            (results_df['dataset'] == dataset)
        ]
        
        if config_df.empty:
            continue
            
        # Create comparison table
        comparison = []
        baseline_row = config_df[config_df['strategy'] == 'baseline'].iloc[0]
        
        for _, row in config_df.iterrows():
            delta_ppl = row['perplexity'] - baseline_row['perplexity']
            delta_acc = row['token_accuracy'] - baseline_row['token_accuracy']
            
            comparison.append({
                'Strategy': row['strategy'],
                'Perplexity': f"{row['perplexity']:.2f}",
                'Δ PPL': f"{delta_ppl:+.2f}",
                'Accuracy': f"{row['token_accuracy']:.4f}",
                'Δ Acc': f"{delta_acc:+.4f}",
                'Speed': f"{row['tokens_per_second']:.1f}",
                'Entropy': f"{row['avg_entropy']:.3f}"
            })
        
        comp_df = pd.DataFrame(comparison)
        print("\n" + comp_df.to_string(index=False))
        
        # Highlight findings
        best_modified = config_df[config_df['strategy'] != 'baseline']['perplexity'].min()
        if best_modified < baseline_row['perplexity']:
            improvement = baseline_row['perplexity'] - best_modified
            best_strategy = config_df[config_df['perplexity'] == best_modified]['strategy'].iloc[0]
            print(f"\n✅ Modified routing improved! Best: {best_strategy} (−{improvement:.2f} PPL)")
        else:
            print(f"\nℹ️  Baseline routing performs best")

## 7. Visualizations

In [ ]:
from IPython.display import Image, display

viz_path = "./two_phase_full_experiment/visualizations/two_phase_comparison.png"
print("📊 Two-Phase Experiment Visualizations:\n")
display(Image(filename=viz_path))

print("\n✅ Visualization includes:")
print("  1. Perplexity comparison (baseline vs modified)")
print("  2. Accuracy comparison")
print("  3. Delta perplexity (improvement/degradation)")
print("  4. Expert utilization (baseline)")
print("  5. Routing entropy comparison")
print("  6. Speed-quality trade-off")

## 8. View Internal Routing Logs

Each configuration has a detailed internal routing log with router_logits for all samples.

In [ ]:
import json

# Example: Load internal routing logs for 8 experts baseline
log_file = "./two_phase_full_experiment/logs/8experts_baseline_wikitext_internal_routing.json"

with open(log_file, 'r') as f:
    internal_logs = json.load(f)

print("=" * 70)
print(f"INTERNAL ROUTING LOGS: 8 Experts Baseline (WikiText)")
print("=" * 70)

print(f"\nConfiguration: {internal_logs['config']}")
print(f"Strategy: {internal_logs['strategy']}")
print(f"Dataset: {internal_logs['dataset']}")
print(f"\nTotal samples logged: {len(internal_logs['samples'])}")

# Show summary
if 'summary' in internal_logs:
    print("\nSummary Statistics:")
    for key, value in internal_logs['summary'].items():
        print(f"  {key}: {value}")

# Show first sample details
if internal_logs['samples']:
    sample = internal_logs['samples'][0]
    print(f"\nFirst Sample Details:")
    print(f"  Sample ID: {sample['sample_id']}")
    print(f"  Num tokens: {sample['num_tokens']}")
    print(f"  Loss: {sample['loss']:.4f}")
    print(f"  Num layers: {len(sample['layers'])}")
    
    if sample['layers']:
        layer = sample['layers'][0]
        print(f"\n  Layer 0:")
        print(f"    Router logits shape: {layer['router_logits_shape']}")
        print(f"    Selected experts (first token): {layer['selected_experts'][0][0]}")
        print(f"    Expert weights (first token): {layer['expert_weights'][0][0]}")

print("\n" + "=" * 70)
print("✅ Internal routing logs contain FULL router_logits data!")
print("=" * 70)

### List All Generated Log Files

In [ ]:
import glob

logs_dir = "./two_phase_full_experiment/logs/"

print("=" * 70)
print("ALL GENERATED LOG FILES")
print("=" * 70)

# Summary logs
summary_logs = sorted(glob.glob(f"{logs_dir}/*.json"))
summary_logs = [f for f in summary_logs if '_internal_routing' not in f]

print(f"\n📄 Summary Logs ({len(summary_logs)} files):")
for log in summary_logs[:10]:  # Show first 10
    print(f"  • {os.path.basename(log)}")
if len(summary_logs) > 10:
    print(f"  ... and {len(summary_logs) - 10} more")

# Internal routing logs
internal_logs = sorted(glob.glob(f"{logs_dir}/*_internal_routing.json"))

print(f"\n🔍 Internal Routing Logs ({len(internal_logs)} files):")
for log in internal_logs[:10]:  # Show first 10
    size_mb = os.path.getsize(log) / (1024 * 1024)
    print(f"  • {os.path.basename(log)} ({size_mb:.2f} MB)")
if len(internal_logs) > 10:
    print(f"  ... and {len(internal_logs) - 10} more")

print(f"\n✅ Total: {len(summary_logs)} summary + {len(internal_logs)} internal routing logs")
print("=" * 70)

## Summary

### What We Accomplished

✅ **Full Two-Phase Experiment**: Analyzed baseline + tested 2 routing modifications  
✅ **Internal Routing Logs**: Saved router_logits for every sample in unique files  
✅ **Comprehensive Analysis**: 45 experiments across 5 expert counts × 3 datasets × 3 strategies  
✅ **Detailed Reports**: Markdown report with tables and recommendations  
✅ **Rich Visualizations**: 6-panel comparison plots  

### Datasets Evaluated

- **WikiText**: Language modeling benchmark
- **LAMBADA**: Word prediction in context
- **HellaSwag**: Commonsense reasoning and sentence completion

### Files Generated

- **90 log files** (45 summary + 45 internal routing)
- **CSV results** with all metrics
- **Markdown report** with detailed analysis
- **PNG/PDF visualizations**

### Next Steps

1. **Analyze internal routing logs** to understand expert selection patterns
2. **Compare router_logits** between baseline and modified routing
3. **Identify patterns** in expert utilization
4. **Design better routing strategies** based on insights

---

**All results persisted in Google Drive!** 🎉